In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import timm
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
import torch

from datasetClass import KSLDataset
from LeNetClass import LeNet
from trainLoop import Trainer

# Загрузка данных

In [3]:
df = pd.read_csv(r"data/Train.csv")

In [5]:
df.groupby(by = "img_IDS").count().value_counts()

Label
1        6249
Name: count, dtype: int64

In [5]:
df

,img_IDS,Label
0,ImageID_33HMDMJ3,Temple
1,ImageID_V5POEX4O,Church
2,ImageID_89CCCCW6,Enough/Satisfied
3,ImageID_GNJXOWX5,Me
4,ImageID_7Q9LOP7R,Love
...,...,...
6244,ImageID_F2HTAA5P,Love
6245,ImageID_0D69ZQ4X,Church
6246,ImageID_E67IZXVJ,Seat
6247,ImageID_ENGL8NP3,You


In [9]:
df['Label'].value_counts()

Label
Enough/Satisfied    695
Seat                695
Mosque              695
Temple              694
Church              694
Love                694
Me                  694
You                 694
Friend              694
Name: count, dtype: int64

In [4]:
df["Label"] = df["Label"].astype("category")
df["target"] = df["Label"].cat.codes

In [26]:
df

,img_IDS,Label,target
0,ImageID_33HMDMJ3,Temple,7
1,ImageID_V5POEX4O,Church,0
2,ImageID_89CCCCW6,Enough/Satisfied,1
3,ImageID_GNJXOWX5,Me,4
4,ImageID_7Q9LOP7R,Love,3
...,...,...,...
6244,ImageID_F2HTAA5P,Love,3
6245,ImageID_0D69ZQ4X,Church,0
6246,ImageID_E67IZXVJ,Seat,6
6247,ImageID_ENGL8NP3,You,8


In [5]:
train_df, val_df = train_test_split(
    df,
    test_size=0.33, random_state=42, stratify=df["Label"])

# Загрузка изображений

In [6]:
images_dir = "data/Images/Images"

train_dataset = KSLDataset(train_df, images_dir)
val_dataset   = KSLDataset(val_df,   images_dir)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# Запуск обучающего цикла LeNet

In [10]:
model = LeNet(in_channels=3,
              num_classes=len(train_df.target.unique()), use_dropout=True)

trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=10)

Epoch 1/10 | train_loss: 2.2005 | val_loss: 2.1977 | val_ROC_AUC: 0.5140
Epoch 2/10 | train_loss: 2.1987 | val_loss: 2.1984 | val_ROC_AUC: 0.5203
Epoch 3/10 | train_loss: 2.1990 | val_loss: 2.1961 | val_ROC_AUC: 0.6027
Epoch 4/10 | train_loss: 2.1795 | val_loss: 2.1400 | val_ROC_AUC: 0.6657
Epoch 5/10 | train_loss: 2.0240 | val_loss: 1.8765 | val_ROC_AUC: 0.7588
Epoch 6/10 | train_loss: 1.8370 | val_loss: 1.7496 | val_ROC_AUC: 0.7910
Epoch 7/10 | train_loss: 1.7244 | val_loss: 1.6758 | val_ROC_AUC: 0.8128
Epoch 8/10 | train_loss: 1.6012 | val_loss: 1.6342 | val_ROC_AUC: 0.8241
Epoch 9/10 | train_loss: 1.5188 | val_loss: 1.5785 | val_ROC_AUC: 0.8347
Epoch 10/10 | train_loss: 1.4429 | val_loss: 1.5252 | val_ROC_AUC: 0.8453
Best val ROC AUC: 0.8453


# Запуск обучающего цикла для resnet18

In [ ]:
model = timm.create_model("resnet18", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_resnet18.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [16]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Model first param device: cuda:0
Epoch 1/4 | train_loss: 1.1830 | val_loss: 0.9181 | val_ROC_AUC: 0.9497
Epoch 2/4 | train_loss: 0.7065 | val_loss: 0.7896 | val_ROC_AUC: 0.9617
Epoch 3/4 | train_loss: 0.4523 | val_loss: 0.9506 | val_ROC_AUC: 0.9421
Epoch 4/4 | train_loss: 0.3433 | val_loss: 0.7743 | val_ROC_AUC: 0.9640
Best val ROC AUC: 0.9640


# Запуск обучающего цикла для efficientnet_b0

In [17]:
model = timm.create_model("efficientnet_b0.ra_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_efficientnet_b0.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [18]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 1.4888 | val_loss: 0.9807 | val_ROC_AUC: 0.9386
Epoch 2/4 | train_loss: 0.8612 | val_loss: 0.7424 | val_ROC_AUC: 0.9647
Epoch 3/4 | train_loss: 0.6155 | val_loss: 0.7187 | val_ROC_AUC: 0.9680
Epoch 4/4 | train_loss: 0.4472 | val_loss: 0.6340 | val_ROC_AUC: 0.9744
Best val ROC AUC: 0.9744


# Запуск обучающего цикла для convnext_tiny

In [ ]:
model = timm.create_model("convnext_tiny.in12k_ft_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_convnext_tiny.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [20]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 2.3058 | val_loss: 2.2180 | val_ROC_AUC: 0.5067
Epoch 2/4 | train_loss: 2.2175 | val_loss: 2.2068 | val_ROC_AUC: 0.5074
Epoch 3/4 | train_loss: 2.2056 | val_loss: 2.2060 | val_ROC_AUC: 0.5181
Epoch 4/4 | train_loss: 2.2046 | val_loss: 2.2012 | val_ROC_AUC: 0.5066
Best val ROC AUC: 0.5181


# Запуск обучающего цикла для densenet

In [21]:
model = timm.create_model("densenet121.ra_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_densenet121.ra_in1k.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [22]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 1.5974 | val_loss: 1.1098 | val_ROC_AUC: 0.9247
Epoch 2/4 | train_loss: 0.8812 | val_loss: 0.8693 | val_ROC_AUC: 0.9557
Epoch 3/4 | train_loss: 0.6050 | val_loss: 0.9139 | val_ROC_AUC: 0.9505
Epoch 4/4 | train_loss: 0.4832 | val_loss: 0.7243 | val_ROC_AUC: 0.9655
Best val ROC AUC: 0.9655


# Проверка разных аугментаций

In [25]:
augs = { "base":
    A.Compose([
    A.Resize(96, 96),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]), # базовая, приведение к одинаковому масштабу
    "ratate" :A.Compose([
        A.Resize(96, 96),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.05,
            rotate_limit=10,
            p=0.5,
            border_mode=cv2.BORDER_REFLECT_101
        ),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]), # отражение+поворот, делает устойчивой к положению объекта
    "color" :A.Compose([
    A.Resize(96, 96),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]), # цвет+яркость, делает устойчивой к освещению/оттенкам
    "cut": A.Compose([
        A.Resize(96, 96),
        A.CoarseDropout(
            num_holes_range=(3, 6),
            hole_height_range=(10, 20),
            hole_width_range=(10, 20),
            fill="random_uniform",
            p=1.0),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]), # закрытие, делает устойчивой к паттернам 
    "mix" : A.Compose([
    A.Resize(96, 96),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.05,
        rotate_limit=10,
        p=0.5,
        border_mode=cv2.BORDER_REFLECT_101
    ),
    A.RandomBrightnessContrast(p=0.5),
    A.CoarseDropout(
        num_holes_range=(3, 6),
        hole_height_range=(10, 20),
        hole_width_range=(10, 20),
        fill="random_uniform",
        p=1.0
    ),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]) # комбинация
}

In [26]:
model = timm.create_model("efficientnet_b0.ra_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_efficientnet_b0.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [28]:
for key in augs:
    train_dataset = KSLDataset(train_df, images_dir, transform = augs[key])
    val_dataset   = KSLDataset(val_df,   images_dir, transform = augs["base"])

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  num_workers=4, pin_memory=True)
    val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    print("-----", key, "-----")
    
    trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
    trainer.fit(num_epochs=4)

----- base -----
Trainer device: cuda
Epoch 1/4 | train_loss: 0.8690 | val_loss: 0.5965 | val_ROC_AUC: 0.9761
Epoch 2/4 | train_loss: 0.4788 | val_loss: 0.5283 | val_ROC_AUC: 0.9816
Epoch 3/4 | train_loss: 0.3706 | val_loss: 0.4890 | val_ROC_AUC: 0.9842
Epoch 4/4 | train_loss: 0.2650 | val_loss: 0.5355 | val_ROC_AUC: 0.9816
Best val ROC AUC: 0.9842
----- ratate -----
Trainer device: cuda
Epoch 1/4 | train_loss: 0.4431 | val_loss: 0.5534 | val_ROC_AUC: 0.9810
Epoch 2/4 | train_loss: 0.3490 | val_loss: 0.6256 | val_ROC_AUC: 0.9756
Epoch 3/4 | train_loss: 0.2987 | val_loss: 0.4461 | val_ROC_AUC: 0.9870
Epoch 4/4 | train_loss: 0.2654 | val_loss: 0.5055 | val_ROC_AUC: 0.9839
Best val ROC AUC: 0.9870
----- color -----
Trainer device: cuda
Epoch 1/4 | train_loss: 0.2351 | val_loss: 0.5857 | val_ROC_AUC: 0.9818
Epoch 2/4 | train_loss: 0.1934 | val_loss: 0.5509 | val_ROC_AUC: 0.9841
Epoch 3/4 | train_loss: 0.1466 | val_loss: 0.5624 | val_ROC_AUC: 0.9847
Epoch 4/4 | train_loss: 0.1435 | val_loss

In [ ]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 1.4888 | val_loss: 0.9807 | val_ROC_AUC: 0.9386
Epoch 2/4 | train_loss: 0.8612 | val_loss: 0.7424 | val_ROC_AUC: 0.9647
Epoch 3/4 | train_loss: 0.6155 | val_loss: 0.7187 | val_ROC_AUC: 0.9680
Epoch 4/4 | train_loss: 0.4472 | val_loss: 0.6340 | val_ROC_AUC: 0.9744
Best val ROC AUC: 0.9744
